In [11]:

# import os
# import joblib
# import numpy as np
# import torch
# from torch.utils.data import DataLoader
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score, accuracy_score
# from sklearn.model_selection import train_test_split
# from tab_transformer_pytorch import TabTransformer
# import os
# import sys
# import json
# import time
# import h5py
# import joblib
# import torch
# import pandas as pd
# import numpy as np

# from datetime import datetime
# from torch import nn
# from torch.utils.data import Dataset, DataLoader
# from torch.optim.lr_scheduler import StepLR
# from sklearn.preprocessing import OrdinalEncoder
# from sklearn.model_selection import train_test_split
# from tab_transformer_pytorch import TabTransformer
# from torch.optim import Adam
# from torch.nn import MSELoss
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# # === Adjust imports based on your project structure ===
# # from running_tf+specdp import CombinedRULModel, RULCombinedDataset, load_from_h5



# H5_PATH = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\data_windows.h5"











# with h5py.File(H5_PATH, "r") as f:
#         X = f["X_windows"][:]
#         y = f["y_labels"][:]
#         vids = f["window_vids"][:]
#         specs = f["specs_per_window"][:]

# class RULCombinedDataset(Dataset):
#     def __init__(self, X, specs, y):
#         self.X = X
#         self.specs = specs
#         self.y = y.reshape(-1, 1)

#     def __len__(self):
#         return len(self.y)

#     def __getitem__(self, i):
#         return (
#             torch.from_numpy(self.specs[i]).long(),
#             torch.from_numpy(self.X[i]).float(),
#             torch.from_numpy(self.y[i]).float()
#         )

# class TimeSeriesEmbedder(nn.Module):
#     def __init__(self, num_features, d_model=128, n_heads=8, num_layers=2, dropout=0.1):
#         super().__init__()
#         self.input_proj = nn.Linear(num_features, d_model)
#         enc_layer = nn.TransformerEncoderLayer(
#             d_model=d_model, nhead=n_heads, dropout=dropout, batch_first=True
#         )
#         self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)

#     def forward(self, x):
#         x = self.input_proj(x)
#         x = self.encoder(x)
#         return x[:, -1, :]
# class CombinedRULModel(nn.Module):
#     def __init__(self, num_sensor_features, context_length, categories, continuous_dim, cont_mean_std=None):
#         super().__init__()
#         self.tf = TimeSeriesEmbedder(num_sensor_features, continuous_dim)
#         if cont_mean_std is None:
#             cont_mean_std = torch.stack([torch.zeros(continuous_dim), torch.ones(continuous_dim)], dim=1)
#         self.tab = TabTransformer(
#             categories=categories,
#             num_continuous=continuous_dim,
#             dim=continuous_dim,
#             dim_out=1,
#             depth=6,
#             heads=8,
#             attn_dropout=0.1,
#             ff_dropout=0.1,
#             mlp_hidden_mults=(4,2),
#             mlp_act=nn.ReLU(),
#             continuous_mean_std=cont_mean_std
#         )

#     def forward(self, x_cat, x_ts):
#         cont = self.tf(x_ts)
#         return self.tab(x_cat, cont)

# # C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script2\artifacts\CombinedRULModel-DP-20250618_160955\checkpoint.pth

# # C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script2\artifacts\CombinedRULModel-NDP-20250616_163702\checkpoint.pth







# # === Paths (adjust as needed) ===
# ENCODER_PATH = r"C:/Users/ASUS/Desktop/SCANIA/2024-34-2/2024-34-2/Important_script/spec_encoder.joblib"
# CHECKPOINT_PATH = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script2\artifacts\CombinedRULModel-DP-20250618_160955\checkpoint.pth"  # <-- UPDATE with actual
# USE_DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # === Load Data ===
# # X, y, _, specs = load_from_h5()
# X_mem, X_nonmem, y_mem, y_nonmem, specs_mem, specs_nonmem = train_test_split(
#     X, y, specs, test_size=0.5, random_state=42
# )



# # === Load Model ===
# encoder = joblib.load(ENCODER_PATH)
# cat_sizes = tuple(len(c) for c in encoder.categories_)

# model = CombinedRULModel(
#     num_sensor_features=X.shape[2],
#     context_length=X.shape[1],
#     categories=cat_sizes,
#     continuous_dim=128
# ).to(USE_DEVICE)

# model.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=USE_DEVICE))
# model.eval()

# # === Collect Per-Sample MSE ===
# def get_losses(X, specs, y):
#     dataset = RULCombinedDataset(X, specs, y)
#     loader = DataLoader(dataset, batch_size=256, shuffle=False)
#     losses = []
#     with torch.no_grad():
#         for xc, xt, yb in loader:
#             xc, xt, yb = xc.to(USE_DEVICE), xt.to(USE_DEVICE), yb.to(USE_DEVICE)
#             preds = model(xc, xt)
#             loss = ((preds - yb) ** 2).squeeze().cpu().numpy()
#             losses.append(loss)
#     return np.concatenate(losses).reshape(-1, 1)

# feat_mem = get_losses(X_mem, specs_mem, y_mem)
# feat_non = get_losses(X_nonmem, specs_nonmem, y_nonmem)

# X_attack = np.vstack([feat_mem, feat_non])
# y_attack = np.concatenate([np.ones(len(feat_mem)), np.zeros(len(feat_non))])

# Xa_train, Xa_test, ya_train, ya_test = train_test_split(
#     X_attack, y_attack, test_size=0.3, random_state=0, stratify=y_attack
# )

#  # === Train Attack Model ===
# attack_model = LogisticRegression(solver="lbfgs")
# attack_model.fit(Xa_train, ya_train)

# # === Evaluate Attack ===
# ya_proba = attack_model.predict_proba(Xa_test)[:, 1]
# ya_preds = attack_model.predict(Xa_test)

# auc = roc_auc_score(ya_test, ya_proba)
# acc = accuracy_score(ya_test, ya_preds)

# print(f"✅ MIA Success Rate (AUC): {auc * 100:.2f}%")
# print(f"✅ MIA Accuracy: {acc * 100:.2f}%")





In [ ]:
npx https://github.com/google-gemini/gemini-cli

# NEEDED MIA ATTACK 

In [2]:
import os
import joblib
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from tab_transformer_pytorch import TabTransformer
import os
import sys
import json
import time
import h5py
import joblib
import torch
import pandas as pd
import numpy as np

from datetime import datetime
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from tab_transformer_pytorch import TabTransformer
from torch.optim import Adam
from torch.nn import MSELoss
from sklearn.metrics import mean_squared_error, mean_absolute_error
# === Adjust imports based on your project structure ===
# from running_tf+specdp import CombinedRULModel, RULCombinedDataset, load_from_h5



H5_PATH = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\data_windows.h5"











with h5py.File(H5_PATH, "r") as f:
        X = f["X_windows"][:]
        y = f["y_labels"][:]
        vids = f["window_vids"][:]
        specs = f["specs_per_window"][:]

class RULCombinedDataset(Dataset):
    def __init__(self, X, specs, y):
        self.X = X
        self.specs = specs
        self.y = y.reshape(-1, 1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return (
            torch.from_numpy(self.specs[i]).long(),
            torch.from_numpy(self.X[i]).float(),
            torch.from_numpy(self.y[i]).float()
        )

class TimeSeriesEmbedder(nn.Module):
    def __init__(self, num_features, d_model=128, n_heads=8, num_layers=2, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(num_features, d_model)
        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=n_heads, dropout=dropout, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)

    def forward(self, x):
        x = self.input_proj(x)
        x = self.encoder(x)
        return x[:, -1, :]
class CombinedRULModel(nn.Module):
    def __init__(self, num_sensor_features, context_length, categories, continuous_dim, cont_mean_std=None):
        super().__init__()
        self.tf = TimeSeriesEmbedder(num_sensor_features, continuous_dim)
        if cont_mean_std is None:
            cont_mean_std = torch.stack([torch.zeros(continuous_dim), torch.ones(continuous_dim)], dim=1)
        self.tab = TabTransformer(
            categories=categories,
            num_continuous=continuous_dim,
            dim=continuous_dim,
            dim_out=1,
            depth=6,
            heads=8,
            attn_dropout=0.1,
            ff_dropout=0.1,
            mlp_hidden_mults=(4,2),
            mlp_act=nn.ReLU(),
            continuous_mean_std=cont_mean_std
        )

    def forward(self, x_cat, x_ts):
        cont = self.tf(x_ts)
        return self.tab(x_cat, cont)

# C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script2\artifacts\CombinedRULModel-DP-20250618_160955\checkpoint.pth

# C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script2\artifacts\CombinedRULModel-NDP-20250616_163702\checkpoint.pth






In [ ]:


# === Paths (adjust as needed) ===
ENCODER_PATH = r"C:/Users/ASUS/Desktop/SCANIA/2024-34-2/2024-34-2/Important_script/spec_encoder.joblib"
CHECKPOINT_PATH = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script2\artifacts\CombinedRULModel-DP-20250618_160955\checkpoint.pth"  # <-- UPDATE with actual
# CHECKPOINT_PATH = r"C:\Users\ASUS\Downloads\best_dp_model_adaptive_largeralpha.pt"  # <-- UPDATE with actual

USE_DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Load Data ===
# X, y, _, specs = load_from_h5()
X_mem, X_nonmem, y_mem, y_nonmem, specs_mem, specs_nonmem = train_test_split(
    X, y, specs, test_size=0.5, random_state=42
)


In [4]:

# === Load Model ===
encoder = joblib.load(ENCODER_PATH)
cat_sizes = tuple(len(c) for c in encoder.categories_)

model = CombinedRULModel(
    num_sensor_features=X.shape[2],
    context_length=X.shape[1],
    categories=cat_sizes,
    continuous_dim=128
).to(USE_DEVICE)

model.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=USE_DEVICE))
model.eval()


C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator OrdinalEncoder from version 1.3.2 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


RuntimeError: Error(s) in loading state_dict for CombinedRULModel:
	Missing key(s) in state_dict: "tf.input_proj.weight", "tf.input_proj.bias", "tf.encoder.layers.0.self_attn.in_proj_weight", "tf.encoder.layers.0.self_attn.in_proj_bias", "tf.encoder.layers.0.self_attn.out_proj.weight", "tf.encoder.layers.0.self_attn.out_proj.bias", "tf.encoder.layers.0.linear1.weight", "tf.encoder.layers.0.linear1.bias", "tf.encoder.layers.0.linear2.weight", "tf.encoder.layers.0.linear2.bias", "tf.encoder.layers.0.norm1.weight", "tf.encoder.layers.0.norm1.bias", "tf.encoder.layers.0.norm2.weight", "tf.encoder.layers.0.norm2.bias", "tf.encoder.layers.1.self_attn.in_proj_weight", "tf.encoder.layers.1.self_attn.in_proj_bias", "tf.encoder.layers.1.self_attn.out_proj.weight", "tf.encoder.layers.1.self_attn.out_proj.bias", "tf.encoder.layers.1.linear1.weight", "tf.encoder.layers.1.linear1.bias", "tf.encoder.layers.1.linear2.weight", "tf.encoder.layers.1.linear2.bias", "tf.encoder.layers.1.norm1.weight", "tf.

1. Collecting Per-Sample MSE Losses



In [ ]:
# _mem->menas taken from my dataset sample
#_nonmem->means taken from non-memory dataset sample


# === Collect Per-Sample MSE ===
def get_losses(X, specs, y):
    dataset = RULCombinedDataset(X, specs, y)
    loader = DataLoader(dataset, batch_size=256, shuffle=False)
    losses = []
    with torch.no_grad():
        for xc, xt, yb in loader:
            xc, xt, yb = xc.to(USE_DEVICE), xt.to(USE_DEVICE), yb.to(USE_DEVICE)
            preds = model(xc, xt)
            loss = ((preds - yb) ** 2).squeeze().cpu().numpy()
            losses.append(loss)
    return np.concatenate(losses).reshape(-1, 1)

feat_mem = get_losses(X_mem, specs_mem, y_mem)
feat_non = get_losses(X_nonmem, specs_nonmem, y_nonmem)

print("Memory Feature Losses:", feat_mem.shape)
print("Non-Memory Feature Losses:", feat_non.shape)



Memory Feature Losses: (3276, 1)
Non-Memory Feature Losses: (3277, 1)


In [5]:
print("Memory Feature Losses:", feat_mem)
print("Non-Memory Feature Losses:", feat_non)

Memory Feature Losses: [[283.5414   ]
 [239.12152  ]
 [  2.9046319]
 ...
 [290.73364  ]
 [430.3982   ]
 [186.13582  ]]
Non-Memory Feature Losses: [[  29.937943]
 [ 268.57318 ]
 [ 112.57375 ]
 ...
 [1673.507   ]
 [  51.18096 ]
 [ 359.92334 ]]



Preparing the Attack Dataset
Features (X_attack): stack the member-losses ABOVE the non-member-losses into one array of shape (total_samples, 1).

Labels (y_attack):1 for every member sample 0 for every non-member sample

stratify=y_attack ensures the same member/non-member ratio in both sets



In [ ]:

# === Prepare Attack Dataset ===
X_attack = np.vstack([feat_mem, feat_non])
y_attack = np.concatenate([np.ones(len(feat_mem)), np.zeros(len(feat_non))])


print("X attack is",X_attack)
print("y attack is",y_attack)
print("X attack shape:", X_attack.shape)
print("y attack shape:", y_attack.shape)

#stratify=y_attack argument in train_test_split ensures that the proportion of member
#and non-member samples remains the same in both the training and test splits of the attack dataset.


Xa_train, Xa_test, ya_train, ya_test = train_test_split(
    X_attack, y_attack, test_size=0.3, random_state=0, stratify=y_attack
)

print("Xa_train shape:", Xa_train.shape)
print("Xa_test shape:", Xa_test.shape)
print("ya_train shape:", ya_train.shape)
print("ya_test shape:", ya_test.shape)


X attack is [[ 283.5414   ]
 [ 239.12152  ]
 [   2.9046319]
 ...
 [1673.507    ]
 [  51.18096  ]
 [ 359.92334  ]]
y attack is [1. 1. 1. ... 0. 0. 0.]


In [7]:
# === Train Attack Model ===
attack_model = LogisticRegression(solver="lbfgs")
attack_model.fit(Xa_train, ya_train)

# === Evaluate Attack ===
ya_proba = attack_model.predict_proba(Xa_test)[:, 1]
ya_preds = attack_model.predict(Xa_test)

auc = roc_auc_score(ya_test, ya_proba)
acc = accuracy_score(ya_test, ya_preds)

print(f"✅ MIA Success Rate (AUC): {auc * 100:.2f}%")
print(f"✅ MIA Accuracy: {acc * 100:.2f}%")


✅ MIA Success Rate (AUC): 49.12%
✅ MIA Accuracy: 49.59%


- Purpose:
The code demonstrates a privacy attack where one investigates if the per-sample loss (MSE) reveals whether a sample was part of the training set.
- Flow:
It first calculates the losses for both membership (seen) and non-membership (unseen) datasets, creates a combined dataset from these loss values, trains a logistic regression model using these simple features, and finally evaluates the attack using accuracy and AUC metrics.
- Implications:
If the attack model achieves a high AUC or accuracy, it indicates that the target model might be leaking information about its training data. This is a key concern in the domain of machine learning privacy.


MIA AUC: Tells you how well the attack distinguishes between member and non-member samples.

MIA Accuracy: Fraction of correctly identified membership status (at default threshold of 0.5).



The “MIA Success Rate” is simply how well your membership‐inference attack distinguishes “members” (records that were in the model’s training set) from “non-members” (records that were held out). In practice it’s reported in one of two ways:

Attack Accuracy

Accuracy=
correctly labeled members
+
correctly labeled non-members
total samples
.
Accuracy= 
total samples
correctly labeled members+correctly labeled non-members
​
 .
If your attack simply guesses “in” or “out” for each sample (typically by thresholding a score at 0.5), the accuracy is the fraction of those guesses that are right.

ROC-AUC (Area Under the Curve)
Rather than pick a fixed threshold, you can plot the receiver-operating characteristic (ROC) curve (true-positive rate vs. false-positive rate as you sweep the threshold). The area under that curve (AUC) is a threshold-independent summary of attack effectiveness.

AUC = 0.5 means “no better than random guessing.”

AUC = 1.0 means a perfect attack.

In MIA work even AUCs of 0.6–0.8 are considered substantial privacy leakage.

You may also report other related figures:

True-Positive Rate (TPR) at a fixed False-Positive Rate (FPR)—e.g. “TPR at FPR=10%”—to show, of the 10% of non-members you falsely flag as members, how many real members you catch.

Precision / Recall / F1-score if you care about imbalanced member vs. non-member ratios.

But in almost every MIA paper the headline “MIA Success Rate” is the attack’s ROC-AUC (or sometimes its plain accuracy) on a held-out test of member vs. non-member samples.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, average_precision_score

# Instantiate
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)

# Train
rf.fit(Xa_train, ya_train)

# Predict
probs = rf.predict_proba(Xa_test)[:,1]
preds = rf.predict(Xa_test)

# Metrics
print("Random Forest Attack:")
print(f" AUC        = {roc_auc_score or MIA Success Rate(ya_test, probs):.3f}")
print(f" Accuracy   = {accuracy_score(ya_test, preds):.3f}")
print(f" Precision  = {precision_score(ya_test, preds):.3f}")
print(f" Recall     = {recall_score(ya_test, preds):.3f}")
print(f" F1-score   = {f1_score(ya_test, preds):.3f}")
print(f" PR-AUC     = {average_precision_score(ya_test, probs):.3f}")


Random Forest Attack:
 AUC        = 0.484
 Accuracy   = 0.488
 Precision  = 0.489
 Recall     = 0.545
 F1-score   = 0.516
 PR-AUC     = 0.489


In [24]:
pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 1.3/150.0 MB 4.8 MB/s eta 0:00:32
    --------------------------------------- 2.6/150.0 MB 5.8 MB/s eta 0:00:26
   - -------------------------------------- 3.9/150.0 MB 5.7 MB/s eta 0:00:26
   - -------------------------------------- 5.8/150.0 MB 6.5 MB/s eta 0:00:23
   -- ------------------------------------- 7.6/150.0 MB 7.0 MB/s eta 0:00:21
   -- ------------------------------------- 9.4/150.0 MB 7.3 MB/s eta 0:00:20
   --- ------------------------------------ 11.3/150.0 MB 7.7 MB/s eta 0:00:19
   --- ------------------------------------ 13.1/150.0 MB 7.8 MB/s eta 0:00:18
   ---- ----------------------------------- 15.2/150.0 MB 8.0 MB/s eta 0:00:17
   ---- ----------------------------------- 17.6/150.0 MB 8.3 MB/s eta 0:00:16
   ----- ---------------------------------- 19.7/150.0 MB 8.5 MB/s eta 0:0

In [25]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, average_precision_score

# Prepare DMatrix
dtrain = xgb.DMatrix(Xa_train, label=ya_train)
dtest  = xgb.DMatrix(Xa_test,  label=ya_test)

# Params
params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "max_depth": 4,
    "eta": 0.1,
    "seed": 0
}

# Train
bst = xgb.train(params, dtrain, num_boost_round=100)

# Predict
probs = bst.predict(dtest)
preds = (probs >= 0.5).astype(int)

# Metrics
print("XGBoost Attack:")
print(f" AUC or MIA Success Rate   = {roc_auc_score(ya_test, probs):.3f}")
print(f" Acc    = {accuracy_score(ya_test, preds):.3f}")
print(f" PR-AUC = {average_precision_score(ya_test, probs):.3f}")


XGBoost Attack:
 AUC or MIA Success Rate   = 0.485
 Acc    = 0.487
 PR-AUC = 0.487


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score

# Note: need probability=True to get predict_proba
svm = SVC(kernel='rbf', C=1.0, probability=True, random_state=0)

svm.fit(Xa_train, ya_train)
probs = svm.predict_proba(Xa_test)[:,1]
preds = svm.predict(Xa_test)

print("SVM Attack:")
print(f" AUC   or MIA Success Rate     = {roc_auc_score(ya_test, probs):.3f}")
print(f" Accuracy = {accuracy_score(ya_test, preds):.3f}")


SVM Attack:
 AUC      = 0.507
 Accuracy = 0.508


In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(64,32), activation='relu', max_iter=200, random_state=0)
mlp.fit(Xa_train, ya_train)

probs = mlp.predict_proba(Xa_test)[:,1]
preds = mlp.predict(Xa_test)

print("MLP Attack:")
print(f" AUC  or MIA Success Rate      = {roc_auc_score(ya_test, probs):.3f}")
print(f" Accuracy = {accuracy_score(ya_test, preds):.3f}")


MLP Attack:
 AUC      = 0.509
 Accuracy = 0.500


In [23]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50,100,200],
    'max_depth': [3,5,10]
}
grid = GridSearchCV(RandomForestClassifier(random_state=0),
                    param_grid, scoring='roc_auc', cv=3, n_jobs=-1)
grid.fit(X_attack, y_attack)

print("Best params:", grid.best_params_)
print("Best CV AUC:", grid.best_score_)


Best params: {'max_depth': 10, 'n_estimators': 50}
Best CV AUC: 0.49900756886928876


In [22]:
from sklearn.ensemble import VotingClassifier

ensemble = VotingClassifier([
    ('lr', LogisticRegression()),
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('mlp', MLPClassifier(hidden_layer_sizes=(32,)))
], voting='soft')

ensemble.fit(Xa_train, ya_train)
probs = ensemble.predict_proba(Xa_test)[:,1]
print("Ensemble AUC:", roc_auc_score(ya_test, probs))


Ensemble AUC: 0.4853718711482797


# OTHER TYPES ATTACK

In [11]:
import os
import joblib
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
from sklearn.model_selection import train_test_split

In [14]:
X_mem, X_non, y_mem, y_non, s_mem, s_non = train_test_split(
    X, y, specs, test_size=0.5, random_state=42
)

In [ ]:
def get_loader(Xa, Sa, ya, batch_size=256):
    ds = RULCombinedDataset(Xa, Sa, ya)
    return DataLoader(ds, batch_size=batch_size, shuffle=False)

# Helper: TPR @ FPR
def tpr_at_fpr(y_true, y_scores, fpr_target=0.1):
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    idx = np.searchsorted(fpr, fpr_target, side="right")
    return tpr[idx] if idx < len(tpr) else tpr[-1]

# 1. Threshold Attack (max confidence)
# def threshold_attack():
#     # Query softmax probabilities
#     mem_probs, non_probs = [], []
#     loader_mem = get_loader(X_mem, s_mem, y_mem)
#     loader_non = get_loader(X_non, s_non, y_non)
#     with torch.no_grad():
#         for loader, store in [(loader_mem, mem_probs), (loader_non, non_probs)]:
#             for xc, xt, yb in loader:
#                 xc, xt = xc.to(DEVICE), xt.to(DEVICE)
#                 logits = model(xc, xt)
#                 # if your model returns a scalar regression, skip; this attack applies to classification
#                 probs = torch.softmax(logits, dim=1)  # classification case
#                 max_conf = probs.max(dim=1)[0].cpu().numpy()
#                 store.append(max_conf)
#     mem_conf = np.concatenate(mem_probs)
#     non_conf = np.concatenate(non_probs)
#     scores = np.concatenate([mem_conf, non_conf])
#     labels = np.concatenate([np.ones_like(mem_conf), np.zeros_like(non_conf)])
#     # Evaluate
#     auc = roc_auc_score(labels, scores)
#     # Choose threshold at equal error rate or 0.5
#     thresh = 0.5
#     preds = (scores >= thresh).astype(int)
#     acc = accuracy_score(labels, preds)
#     tpr = tpr_at_fpr(labels, scores, fpr_target=0.1)
#     print("Threshold Attack:")
#     print(f" AUC={auc:.3f}, Acc={acc:.3f}, TPR@FPR=0.1={tpr:.3f}")

# # 2. Neural Attack on full probability vector
# def neural_prob_attack():
#     # Collect full softmax vectors
#     mem_vecs, non_vecs = [], []
#     for loader, store in [(get_loader(X_mem, s_mem, y_mem), mem_vecs),
#                           (get_loader(X_non, s_non, y_non), non_vecs)]:
#         with torch.no_grad():
#             for xc, xt, yb in loader:
#                 xc, xt = xc.to(DEVICE), xt.to(DEVICE)
#                 logits = model(xc, xt)
#                 probs = torch.softmax(logits, dim=1).cpu().numpy()
#                 store.append(probs)
#     Xp = np.vstack([np.concatenate(mem_vecs), np.concatenate(non_vecs)])
#     yp = np.concatenate([np.ones(len(mem_vecs)*loader.batch_size), np.zeros(len(non_vecs)*loader.batch_size)])
#     # Attack train/test split
#     Xa_tr, Xa_te, ya_tr, ya_te = train_test_split(Xp, yp, test_size=0.3, random_state=0, stratify=yp)
#     atk = MLPClassifier(hidden_layer_sizes=(50,), max_iter=200)
#     atk.fit(Xa_tr, ya_tr)
#     probs = atk.predict_proba(Xa_te)[:,1]
#     preds = atk.predict(Xa_te)
#     auc = roc_auc_score(ya_te, probs)
#     acc = accuracy_score(ya_te, preds)
#     tpr = tpr_at_fpr(ya_te, probs)
#     print("Neural Prob Attack:")
#     print(f" AUC={auc:.3f}, Acc={acc:.3f}, TPR@FPR=0.1={tpr:.3f}")

# 3. Representation Attack (extract cont embedding)
def rep_attack():
    mem_emb, non_emb = [], []
    loader_mem = get_loader(X_mem, s_mem, y_mem)
    loader_non = get_loader(X_non, s_non, y_non)
    with torch.no_grad():
        for loader, store in [(loader_mem, mem_emb), (loader_non, non_emb)]:
            for xc, xt, yb in loader:
                xc, xt = xc.to(DEVICE), xt.to(DEVICE)
                emb = model.tf(xt)  # cont embedding
                store.append(emb.cpu().numpy())
    Xr = np.vstack([np.concatenate(mem_emb), np.concatenate(non_emb)])
    yr = np.concatenate([np.ones(len(mem_emb)*loader.batch_size), np.zeros(len(non_emb)*loader.batch_size)])
    Xr_tr, Xr_te, yr_tr, yr_te = train_test_split(Xr, yr, test_size=0.3, random_state=0, stratify=yr)
    atk = LogisticRegression(max_iter=200)
    atk.fit(Xr_tr, yr_tr)
    probs = atk.predict_proba(Xr_te)[:,1]
    preds = atk.predict(Xr_te)
    auc = roc_auc_score(yr_te, probs)
    acc = accuracy_score(yr_te, preds)
    tpr = tpr_at_fpr(yr_te, probs)
    print("Representation Attack:")
    print(f" AUC={auc:.3f}, Acc={acc:.3f}, TPR@FPR=0.1={tpr:.3f}")

# 4. Gradient-Norm Attack (white-box)
def grad_norm_attack():
    mem_norms, non_norms = [], []
    loss_fn = torch.nn.MSELoss(reduction='sum')
    for loader, store in [(get_loader(X_mem, s_mem, y_mem), mem_norms),
                          (get_loader(X_non, s_non, y_non), non_norms)]:
        for xc, xt, yb in loader:
            xc, xt, yb = xc.to(DEVICE), xt.to(DEVICE), yb.to(DEVICE)
            model.zero_grad()
            preds = model(xc, xt)
            loss = loss_fn(preds, yb)
            loss.backward()
            # grad norm aggregated over parameters
            total_norm = 0.0
            for p in model.parameters():
                if p.grad is not None:
                    total_norm += p.grad.detach().norm().item()**2
            total_norm = total_norm**0.5
            store.append(total_norm)
    nm = np.array(mem_norms)
    nnm = np.array(non_norms)
    scores = np.concatenate([nm, nnm])
    labels = np.concatenate([np.ones_like(nm), np.zeros_like(nnm)])
    auc = roc_auc_score(labels, scores)
    thresh = np.median(scores)
    preds = (scores >= thresh).astype(int)
    acc = accuracy_score(labels, preds)
    tpr = tpr_at_fpr(labels, scores)
    print("Gradient-Norm Attack:")
    print(f" AUC={auc:.3f}, Acc={acc:.3f}, TPR@FPR=0.1={tpr:.3f}")

In [ ]:
if __name__ == "__main__":
    # threshold_attack()
    # neural_prob_attack()
    # rep_attack()
    grad_norm_attack()


Threshold Attack:
 AUC=0.500, Acc=0.500, TPR@FPR=0.1=1.000
Gradient-Norm Attack:
 AUC=0.337, Acc=0.462, TPR@FPR=0.1=0.000


Threshold Attack (using max predicted probability)

Neural Attack (MLP on full softmax vector)

Representation Attack (logistic regression on the model’s time-series embedding)

Gradient-Norm Attack (white-box, using per-sample gradient norms)

Each attack prints AUC, Accuracy, and TPR at FPR=0.1 so you can directly compare their effectiveness.

# ADVANCED MIA ATTACK

black-box (loss), gray-box (embedding), and white-box (gradient) features along with time-series-specific seasonality/trend features,

In [ ]:
import os
import joblib
import h5py
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score,
    recall_score, f1_score, roc_curve
)
from sklearn.model_selection import train_test_split
from tab_transformer_pytorch import TabTransformer

# === Paths ===
H5_PATH         = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\data_windows.h5"
ENCODER_PATH    = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\spec_encoder.joblib"
CHECKPOINT_PATH = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script2\artifacts\CombinedRULModel-DP-20250618_160955\checkpoint.pth"
DEVICE          = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Load windows dataset ===
with h5py.File(H5_PATH, "r") as f:
    X = f["X_windows"][:]
    y = f["y_labels"][:]
    specs = f["specs_per_window"][:]

# === Dataset & Model definitions (as before) ===
class RULCombinedDataset(Dataset):
    def __init__(self, X, specs, y):
        self.X = X; self.specs = specs; self.y = y.reshape(-1,1)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, i):
        return (
            torch.from_numpy(self.specs[i]).long(),
            torch.from_numpy(self.X[i]).float(),
            torch.from_numpy(self.y[i]).float()
        )

class TimeSeriesEmbedder(nn.Module):
    def __init__(self, num_features, d_model=128, n_heads=8, num_layers=2, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(num_features, d_model)
        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=n_heads, dropout=dropout, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
    def forward(self, x):
        x = self.input_proj(x)
        x = self.encoder(x)
        return x[:, -1, :]

class CombinedRULModel(nn.Module):
    def __init__(self, num_sensor_features, context_length, categories, continuous_dim, cont_mean_std=None):
        super().__init__()
        self.tf = TimeSeriesEmbedder(num_sensor_features, continuous_dim)
        if cont_mean_std is None:
            cont_mean_std = torch.stack([torch.zeros(continuous_dim), torch.ones(continuous_dim)], dim=1)
        self.tab = TabTransformer(
            categories=categories,
            num_continuous=continuous_dim,
            dim=continuous_dim,
            dim_out=1,
            depth=6, heads=8,
            attn_dropout=0.1, ff_dropout=0.1,
            mlp_hidden_mults=(4,2), mlp_act=nn.ReLU(),
            continuous_mean_std=cont_mean_std
        )
    def forward(self, x_cat, x_ts):
        cont = self.tf(x_ts)
        return self.tab(x_cat, cont)

# === Split members/non-members ===
X_mem, X_non, y_mem, y_non, s_mem, s_non = train_test_split(
    X, y, specs, test_size=0.5, random_state=42
)

# === Load model ===
encoder = joblib.load(ENCODER_PATH)
cat_sizes = tuple(len(c) for c in encoder.categories_)

model = CombinedRULModel(
    num_sensor_features=X.shape[2],
    context_length=X.shape[1],
    categories=cat_sizes,
    continuous_dim=128
).to(DEVICE)

model.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=DEVICE))
model.eval()

# === Helpers ===
def get_loader(Xa, Sa, ya, bs=256):
    return DataLoader(RULCombinedDataset(Xa, Sa, ya), batch_size=bs, shuffle=False)



# given true labels and attack‐scores, returns the True-Positive Rate when the 
# False-Positive Rate is at most fpr_target (e.g. 10%).
def tpr_at_fpr(y_true, y_scores, fpr_target=0.1):
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    idx = np.searchsorted(fpr, fpr_target, side="right")
    return tpr[idx] if idx < len(tpr) else tpr[-1]




# === Feature Extractors ===

# Loss (black-box)
# xc: categorical specs; xt: time-series window; yb: true RUL.
# Computes per-sample MSE as a 1D feature.
# 1) Loss (MSE)
def feat_loss(xc, xt, yb):
    with torch.no_grad():
        preds = model(xc, xt)
    return ((preds - yb)**2).cpu().numpy().reshape(-1,1)


# Embedding Norm (gray-box)
# Runs only the TimeSeriesEmbedder on the window.Takes the ℓ₂‐norm of each 
# embedding vector (detached so that gradients aren’t tracked).
# 2) Embedding norm (detached)
def feat_emb_norm(xt):
    emb = model.tf(xt)
    return torch.norm(emb.detach(), dim=1).cpu().numpy().reshape(-1,1)


#  Gradient Norm (white-box)
# Computes the gradient of the loss w.r.t. all parameters, then aggregates their 
# norms.Broadcasts the same gradient‐norm scalar across the batch as a feature.

# 3) Gradient norm (white-box)
# 3) Gradient norm (white‐box), fixed to use .item()
def feat_grad_norm(xc, xt, yb):
    model.zero_grad()
    preds = model(xc, xt)
    loss = torch.nn.functional.mse_loss(preds, yb, reduction="sum")
    loss.backward()
    total_sq = 0.0
    for p in model.parameters():
        if p.grad is not None:
            # use .item() to get Python float
            norm = p.grad.detach().norm().item()
            total_sq += norm**2
    total = np.sqrt(total_sq)
    # return shape (batch,1): we broadcast the same value across the batch
    return np.full((xc.size(0), 1), total)


# Seasonality: takes the first 5 Fourier magnitudes for each series in the window.
# Trend: fits a 3rd‐degree polynomial to each feature’s time‐series via least 
# squares; concatenates all coefficient vectors.

# 4) Trend & Seasonality
def feat_trend_season(xt_np):
    B, T, M = xt_np.shape
    Xf = np.fft.rfft(xt_np, axis=1)
    season = np.abs(Xf[:, :5, :]).reshape(B, -1)
    t = np.linspace(0,1,T)
    V = np.vander(t, N=4, increasing=True)
    coeffs = []
    for b in range(B):
        cs = []
        for m in range(M):
            c, *_ = np.linalg.lstsq(V, xt_np[b,:,m], rcond=None)
            cs.append(c)
        coeffs.append(np.concatenate(cs))
    trend = np.stack(coeffs, axis=0)
    return np.hstack([season, trend])





# Loops over both member and non-member splits.
# Extracts all four features for each batch.
# Stacks them horizontally into a single feature vector per sample.
# Labels members as 1, non-members as 0.
# === Collect features + labels ===
def collect_features(Xa, Sa, ya, label):
    loader = get_loader(Xa, Sa, ya)
    feats, labs = [], []
    for xc, xt, yb in loader:
        xc, xt, yb = xc.to(DEVICE), xt.to(DEVICE), yb.to(DEVICE)
        xt_np = xt.cpu().numpy()
        L  = feat_loss(xc, xt, yb)
        E  = feat_emb_norm(xt)
        G  = feat_grad_norm(xc, xt, yb)
        TS = feat_trend_season(xt_np)
        batch_f = np.hstack([L, E, G, TS])
        feats.append(batch_f)
        labs.extend([label]*batch_f.shape[0])
    return np.vstack(feats), np.array(labs)




In [12]:
Fm, ym = collect_features(X_mem, s_mem, y_mem, 1)
Fn, yn = collect_features(X_non, s_non, y_non, 0)

X_attack = np.vstack([Fm, Fn])
y_attack = np.concatenate([ym, yn])

# === Split & train attack ===
Xa_tr, Xa_te, ya_tr, ya_te = train_test_split(
    X_attack, y_attack, test_size=0.3,
    random_state=0, stratify=y_attack
)


In [13]:

atk = LogisticRegression(max_iter=500)
atk.fit(Xa_tr, ya_tr)

probs = atk.predict_proba(Xa_te)[:,1]
preds = atk.predict(Xa_te)

# === Metrics ===
auc  = roc_auc_score(ya_te, probs)
acc  = accuracy_score(ya_te, preds)
tpr1 = tpr_at_fpr(ya_te, probs, fpr_target=0.1)
prec = precision_score(ya_te, preds)
rec  = recall_score(ya_te, preds)
f1   = f1_score(ya_te, preds)

print("=== Extended MIA Results ===")
print(f"AUC         : {auc:.4f}")
print(f"Accuracy    : {acc:.4f}")
print(f"TPR@FPR=0.1 : {tpr1:.4f}")
print(f"Precision   : {prec:.4f}")
print(f"Recall      : {rec:.4f}")
print(f"F1-score    : {f1:.4f}")

=== Extended MIA Results ===
AUC         : 0.5742
Accuracy    : 0.5331
TPR@FPR=0.1 : 0.1211
Precision   : 0.5378
Recall      : 0.4700
F1-score    : 0.5016
